In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('MovieRecomender') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

<SparkContext master=local[*] appName=MovieRecomender>
Spark App Name : MovieRecomender


In [3]:
import os 
os.chdir ('/home/jovyan/work/MovieRecomender')
#change to your working directory
%pwd

'/home/jovyan/work/MovieRecomender'

In [4]:
df_m = pd.read_csv("data/movies.dat", engine='python', sep='::', names=["MovieID", "Title", "Genres"],encoding='ISO-8859-1')
df_m = spark.createDataFrame(df_m)

df_r = pd.read_csv("data/ratings.dat", engine='python', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])
df_r = spark.createDataFrame(df_r)

df_u = pd.read_csv("data/users.dat", engine='python', sep='::', names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
df_u = spark.createDataFrame(df_u)

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
df_m.printSchema()
df_m.show(3)

root
 |-- MovieID: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genres: string (nullable = true)

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



In [ ]:
df_r.printSchema()
df_r.show(3)

root
 |-- UserID: long (nullable = true)
 |-- MovieID: long (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Timestamp: long (nullable = true)

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
+------+-------+------+---------+
only showing top 3 rows



In [ ]:
df_u.printSchema()
df_u.show(3)

root
 |-- UserID: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Occupation: long (nullable = true)
 |-- Zip-code: string (nullable = true)

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
+------+------+---+----------+--------+
only showing top 3 rows



In [ ]:
df=df_m.join(df_r,"MovieID","inner").join(df_u,"UserID","inner")
df.printSchema()
df.show(3)

root
 |-- UserID: long (nullable = true)
 |-- MovieID: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Timestamp: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Occupation: long (nullable = true)
 |-- Zip-code: string (nullable = true)

+------+-------+--------------------+--------------------+------+---------+------+---+----------+--------+
|UserID|MovieID|               Title|              Genres|Rating|Timestamp|Gender|Age|Occupation|Zip-code|
+------+-------+--------------------+--------------------+------+---------+------+---+----------+--------+
|     5|     29|City of Lost Chil...|    Adventure|Sci-Fi|     5|978245065|     M| 25|        20|   55455|
|     6|   1806|       Paulie (1998)|Adventure|Childre...|     3|978236876|     F| 50|         9|   55117|
|     7|    474|In the Line of Fi...|     Action|Thriller|     5|978234842|     M| 35

In [ ]:
df=df.select(["UserID","MovieID","Rating"])
df.na.drop()
df.show(3)

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
(train, test) = df.randomSplit([0.8, 0.2])

In [ ]:
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False)

In [ ]:
grid_search = ParamGridBuilder().addGrid(als.rank, [10, 25, 50 ,75 , 100]  ).addGrid(als.maxIter,[5, 10, 20, 40, 80, 160]  ).addGrid(als.regParam, [.001,.005,.01, .05, .1, .5] ).build()

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

In [ ]:
cv = CrossValidator(estimator=als, estimatorParamMaps=grid_search, evaluator=evaluator, numFolds=5)

In [ ]:
cv_fitted=cv.fit(train)

IllegalArgumentException: rating does not exist. Available: UserID, MovieID, Rating, CrossValidator_99d38bf33229_rand, prediction